# Hybrid CNN-RNN Model using TensorFlow

### Importing Clean Data & Dependencies

In [7]:
from data_clean import df_prediction as df
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam

### Preparing Data for Model Training

Reformatting Date Column

In [8]:
df = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.loc[:, 'Day'] = df['Date'].dt.day
df.loc[:, 'Month'] = df['Date'].dt.month
df.loc[:, 'Year'] = df['Date'].dt.year
df = df.drop(columns=['Date'])
df


,Latitude,Longitude,Depth,Magnitude,Day,Month,Year
0,-9.18,119.06,10,4.9,1,11,2008
1,-6.55,129.64,10,4.6,1,11,2008
2,-7.01,106.63,121,3.7,1,11,2008
3,-3.30,127.85,10,3.2,1,11,2008
4,-6.41,129.54,70,4.3,1,11,2008
...,...,...,...,...,...,...,...
81218,-8.02,121.81,69,3.4,10,1,2022
81219,1.50,127.85,10,3.7,10,1,2022
81220,1.50,127.86,10,3.0,10,1,2022
81221,1.48,127.86,10,2.7,10,1,2022


Preparing X and y

In [9]:
feature_columns = ['Latitude', 'Longitude', 'Depth', 'Day', 'Month', 'Year']
target_columns = ['Magnitude', 'Latitude', 'Longitude']

In [10]:
X = df[feature_columns]
y = df[target_columns]

Converting Date to Ordinal Format

In [11]:
X['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
X['Date'] = X['Date'].map(pd.Timestamp.toordinal)

# drop the original Day, Month, and Year columns
X = X.drop(['Day', 'Month', 'Year'], axis=1)

Normalizing X

In [12]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Splitting the data

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### Building the Model

In [14]:
input_layer = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(input_layer)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)

Creating one output layer for each target

In [15]:
magnitude_output = Dense(1, name='magnitude')(x)
latitude_output = Dense(1, name='latitude')(x)
longitude_output = Dense(1, name='longitude')(x)

In [16]:
model = Model(inputs=input_layer, outputs=[magnitude_output, latitude_output, longitude_output])

Compiling the model

In [17]:
model.compile(optimizer=Adam(), loss='mse')

### Training the Model

In [18]:
model.fit(X_train, [y_train['Magnitude'], y_train['Latitude'], y_train['Longitude']], 
          epochs=500, batch_size=32, validation_split=0.2)

Epoch 1/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 435us/step - loss: 4557.2104 - val_loss: 25.8817
Epoch 2/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step - loss: 344.7773 - val_loss: 17.3770
Epoch 3/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 391us/step - loss: 183.3144 - val_loss: 18.7598
Epoch 4/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 396us/step - loss: 69.1075 - val_loss: 16.8620
Epoch 5/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 390us/step - loss: 21.0433 - val_loss: 11.0664
Epoch 6/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 391us/step - loss: 11.5568 - val_loss: 3.6550
Epoch 7/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 391us/step - loss: 9.3260 - val_loss: 3.6872
Epoch 8/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 387us/step - loss: 8.1577 - val_loss: 5.2639
Epoch 9/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 400us/step - loss: 7.3030 - val_loss: 4.0364
Epoch 10/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 389us/step - loss: 6.6734 - val_loss: 4.6156
Epoch 11/500
1625/1625 ━━━━━━━━━━━━━━━━━━━━ 1s 394us/step - loss: 6.2529 -

### Results

In [20]:
predictions = model.predict(X_test)

magnitude_predictions = predictions[0]
latitude_predictions = predictions[1]
longitude_predictions = predictions[2]


508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 270us/step


In [21]:
print(f"Magnitude = {magnitude_predictions[0][0]}")
print(f"Latitude = {latitude_predictions[0][0]}")
print(f"Longitude = {longitude_predictions[0][0]}")

Magnitude = 3.8346524238586426
Latitude = 0.6393423080444336
Longitude = 127.62979888916016


### Evaluating the Model

In [19]:
model.evaluate(X_test, [y_test['Magnitude'], y_test['Latitude'], y_test['Longitude']])

508/508 ━━━━━━━━━━━━━━━━━━━━ 0s 193us/step - loss: 8.7447


8.940865516662598